In [8]:
from scripts.analysis.data_generation import determine_correlation_matrix, sample_realworld_methyl_val, beta_to_m, \
    simulate_methyl_data, load_realworld_data
import numpy as np
import pandas as pd

In [9]:
def proportion_above_threshold(array, threshold):
   return np.sum(array > threshold) / array.size

def proportion_below_threshold(array, threshold):
    return np.sum(array < threshold) / array.size

realworld_data_path = "../data/realworld_methyl_beta.h5"
realworld_data = load_realworld_data(file_path=realworld_data_path)
base_config = {
    'n_sites': 10000,
    'n_observations': 100,
    'dependencies': 1,
    'bin_size_ratio': 0.3,
    'correlation_strength': "high"
}
high_corr_config = base_config.copy()
high_corr_smaller_bin_config = base_config.copy()
high_corr_smaller_bin_config['bin_size_ratio'] = 0.2
medium_corr_config = base_config.copy()
medium_corr_config['correlation_strength'] = "medium"
medium_corr_smaller_bin_config = medium_corr_config.copy()
medium_corr_smaller_bin_config['bin_size_ratio'] = 0.2
config_dicts = [high_corr_config, high_corr_smaller_bin_config, medium_corr_config, medium_corr_smaller_bin_config]

In [13]:
n_replicates = 1
results_list = []
corr_threshold = 0.4
for config in config_dicts:
    if config['correlation_strength'] == 'medium':
        corr_coef_dist = [(-0.6, -0.4), (-0.1, 0.1), (0.4, 0.6)]
    elif config['correlation_strength'] == 'high':
        corr_coef_dist = [(-0.85, -0.7), (-0.1, 0.1), (0.7, 0.85)]
    sub_results_dict = {"correlation_strength": config['correlation_strength'], "bin_size_ratio": config['bin_size_ratio'],
                    'above_threshold': np.array([]), 'below_threshold': np.array([])}
    for i  in range(n_replicates):
        simulated_data = simulate_methyl_data(realworld_data, config['n_sites'], config['n_observations'],
                                                      config['dependencies'],
                                                      int(config['bin_size_ratio'] * config['n_sites']), corr_coef_dist)
        corr_mat = determine_correlation_matrix(simulated_data)
        corr_mat = np.triu(corr_mat, k=1)
        thresholded_corr_mat = corr_mat[np.abs(corr_mat) > corr_threshold]
        quantiles = np.percentile(thresholded_corr_mat, np.arange(0, 101, 25))
        intervals = [(round(quantiles[i], 2), round(quantiles[i + 1], 2)) for i in range(len(quantiles) - 1)]
        sub_results_dict['above_threshold'] = np.append(sub_results_dict['above_threshold'], proportion_above_threshold(corr_mat, corr_threshold))
        sub_results_dict['below_threshold'] = np.append(sub_results_dict['below_threshold'], proportion_below_threshold(corr_mat, -(corr_threshold)))
        sub_results_dict['intervals'] = intervals
    results_list.append(sub_results_dict)
sub_results_dict = {"correlation_strength": "not_applicable", "bin_size_ratio": "not_applicable",
                    'above_threshold': np.array([]), 'below_threshold': np.array([])}
for i in range(n_replicates):
    semi_real_world_data = sample_realworld_methyl_val(n_sites=10000, realworld_data=realworld_data)
    semi_real_world_data = beta_to_m(methyl_beta_values=semi_real_world_data)
    np.random.shuffle(semi_real_world_data)
    corr_mat = determine_correlation_matrix(semi_real_world_data)
    corr_mat = np.triu(corr_mat, k=1)
    thresholded_corr_mat = corr_mat[np.abs(corr_mat) > corr_threshold]
    quantiles = np.percentile(thresholded_corr_mat, np.arange(0, 101, 25))
    intervals = [(round(quantiles[i], 2), round(quantiles[i + 1], 2)) for i in range(len(quantiles) - 1)]
    sub_results_dict['above_threshold'] = np.append(sub_results_dict['above_threshold'], proportion_above_threshold(corr_mat, 0.3))
    sub_results_dict['below_threshold'] = np.append(sub_results_dict['below_threshold'], proportion_below_threshold(corr_mat, -0.3))
    sub_results_dict['intervals'] = intervals
results_list.append(sub_results_dict)



In [14]:

for every_dict in results_list:
    every_dict['above_threshold_mean'] = every_dict['above_threshold'].mean()
    every_dict['above_threshold_std'] = every_dict['above_threshold'].std()
    every_dict['below_threshold_mean'] = every_dict['below_threshold'].mean()
    every_dict['below_threshold_std'] = every_dict['below_threshold'].std()
    del every_dict['above_threshold']
    del every_dict['below_threshold']
# make a dataframe of results_list by concatenating all the dicts in the list
results_df = pd.DataFrame(results_list)

In [17]:
results_df


,correlation_strength,bin_size_ratio,intervals,above_threshold_mean,above_threshold_std,below_threshold_mean,below_threshold_std
0,high,0.3,"[(-0.91, 0.58), (0.58, 0.64), (0.64, 0.71), (0...",0.088839,0.000341,0.000035,3.680882e-07
1,high,0.2,"[(-0.91, 0.54), (0.54, 0.59), (0.59, 0.64), (0...",0.057659,0.001925,0.000049,9.720425e-07
2,medium,0.3,"[(-0.69, 0.41), (0.41, 0.43), (0.43, 0.46), (0...",0.004551,0.000493,0.000030,1.968829e-06
3,medium,0.2,"[(-0.75, 0.41), (0.41, 0.43), (0.43, 0.45), (0...",0.003893,0.001382,0.000041,1.069964e-06
4,not_applicable,not_applicable,"[(-0.95, 0.42), (0.42, 0.48), (0.48, 0.57), (0...",0.108272,0.001094,0.024189,5.202790e-04
